<a href="https://colab.research.google.com/github/Zooshi/llama2local/blob/main/Llama2_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Aug 25 06:46:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.30GHz
    CPU family:          6
    Model:               63
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4599.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscal
                         l nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopo
                         logy nonstop_tsc cpuid tsc_known_freq pni pclmulqdq sss
                         e3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes 
                         xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_s

In [3]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.7 MB/s eta 0:00:00
  Created wheel for ctransformers: filename=ctransformers-0.2.24-cp310-cp310-linux_x86_64.whl size=2312845 sha256=e993a3d991455ca0c4888797449a48a3051a6ec48f281588338af6485d6e02e4
  Stored in directory: /root/.cache/pip/wheels/95/ba/12/d5c2185a1a114d3cf84b3e89a51339ca05f9a17c3f1df6597c
Successfully built ctransformers


In [4]:
model_id = "TheBloke/Llama-2-7B-chat-GGML"

In [7]:
from ctransformers import AutoModelForCausalLM
config = {'max_new_tokens': 256, 'repetition_penalty': 1.1, 'temperature': 0.1, 'stream': True}

llm = AutoModelForCausalLM.from_pretrained(model_path_or_repo_id=model_id, model_type="llama", gpu_layers=110, **config)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
prompt = """Is a higher rep range in weight training better for loosing weigtht. List pro and contra arguments"""

In [9]:
tokens = llm.tokenize(prompt)
tokens

[1,
 3624,
 263,
 6133,
 1634,
 3464,
 297,
 7688,
 6694,
 2253,
 363,
 658,
 14556,
 591,
 335,
 386,
 29873,
 29889,
 2391,
 410,
 322,
 6761,
 6273]

In [12]:
llm.detokenize([6273])

' arguments'

In [13]:
llm(prompt, stream = False)

'\n\nAnswer: A higher rep range in weight training can be beneficial for losing weight, but it depends on several factors. Here are some pros and cons of using a higher rep range for weight loss:\n\nPros:\n\n1. Increased muscle confusion: Higher rep ranges (10-15 reps) can lead to more muscle damage and disruption of muscle fibers, which can promote hypertrophy and fat loss.\n2. Greater muscle activation: Higher rep ranges may activate more muscle mass, leading to increased energy expenditure and fat loss.\n3. Improved insulin sensitivity: Studies have shown that higher rep ranges (10-15 reps) can improve insulin sensitivity, which is essential for weight loss.\n4. Reduced risk of overtraining: Higher rep ranges may reduce the risk of overtraining, as they require less volume and intensity compared to lower rep ranges.\n\nCons:\n\n1. Lower strength gains: Higher rep ranges (10-15 reps) typically result in lower strength gains compared to lower rep ranges (3-5 reps).\n2'

In [16]:
prompt2 = """Welche gesundheitlichen Vorteile hat Cardiotraining für den Körper? Liste 5 Stück auf """

In [17]:
llm(prompt2, stream=False)

'5 Punkten.\n\n1. Cardiotraining kann dazu beitragen, die Blutdruckerhöhung zu reduzieren.\n2. Es kann dazu beitragen, das Risiko von Herzinfarkt und Schlagaderkrise zu verringern.\n3. Ein regelmäßiger Trainingsplan kann dazu beitragen, die Einnahmen von Cholestin in der Blutserumsenke zu reduzieren.\n4. Es kann dazu beitragen, das Risiko von Herz-Kreislauf-Erkrankungen wie Herzinfarkt und Schlagaderkrise zu verringern.\n5. Cardiotraining kann dazu beitragen, die Fitness und die Stamina des Körpers zu verbessern.'

In [26]:
# n = ""
# for i in llm(prompt2, stream=True):
#   n = n + i
#   print(n,end='',flush=True)


In [23]:
# LlAMA-2-13b-chat execution
import time
start = time.time()
NUM_TOKENS=0
print('-'*4+'Start Generation'+'-'*4)
for token in llm.generate(tokens):
    print(llm.detokenize(token), end='', flush=True)
    NUM_TOKENS+=1
time_generate = time.time() - start
print('\n')
print('-'*4+'End Generation'+'-'*4)
print(f'Num of generated tokens: {NUM_TOKENS}')
print(f'Time for complete generation: {time_generate}s')
print(f'Tokens per secound: {NUM_TOKENS/time_generate}')
print(f'Time per token: {(time_generate/NUM_TOKENS)*1000}ms')

----Start Generation----


Answer: A higher rep range in weight training is generally considered to be more effective for losing weight, but there are some factors to consider when deciding on the best approach. Here are some pros and cons of a higher rep range for weight loss:

Pros:

1. Increased muscle confusion: Higher rep ranges (10-15 reps) have been shown to induce more muscle confusion, which can lead to greater gains in muscle mass and strength. This is because the higher rep range stimulates the muscles more intensely, causing them to adapt and grow more effectively.
2. Better for hypertrophy: Higher rep ranges are generally better for building muscle mass and hypertrophy, as they allow for more volume and stress on the muscles. This can lead to greater gains in muscle size and strength over time.
3. More metabolically demanding: Higher rep ranges tend to be more metabolically demanding, meaning that your body will burn more calories at rest and during exercise, which can hel

KeyboardInterrupt: ignored

In [27]:
!ls

sample_data


# Langchain

In [28]:
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00


In [29]:
from langchain.llms import CTransformers

In [30]:
llm2 = CTransformers(model = model_id,model_type='llama',lib='avx')

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
llm2("Where is the puck going for Generative AI")

'?\n\nThe generative AI field is in the early stages of research, but it has the potential to revolutionize many industries. The technology uses complex algorithms and models to generate new data, images, and content based on patterns found in existing data. Here are some areas where the puck might be going for generative AI:\n1. Entertainment: Generative AI could create new music, videos, and stories based on existing works. For instance, an AI system could analyze a novel or a movie script and generate a sequel or a spin-off without any human intervention.\n2. Advertising and Marketing: With the ability to produce diverse content quickly and efficiently, generative AI could revolutionize the advertising industry by creating personalized ads that are tailored to specific audiences.\n3. Fashion: Generative AI can generate new fashion designs based on patterns found in existing styles, which could help designers and stylists save time and resources.\n4. Medicine: Generative AI could be 